In [35]:
from langchain_community.document_loaders import TextLoader

In [36]:
loader = TextLoader("speech.txt")
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='\nThe world is changed.\n\nI feel it in the water.\n\n\nI feel it in the earth.\n\n\nI smell it in the air.\n\nMuch that once was is lost, for none now live who remember it.\n\nIt began with the forging of the Great Rings. Three were given to the Elves, immortal, wisest and fairest of all beings. Seven to the Dwarf-Lords, great miners and craftsmen of the mountain halls. And nine, nine rings were gifted to the race of Men, who above all else desire power. For within these rings was bound the strength and the will to govern each race. But they were all of them deceived, for another ring was made. Deep in the land of Mordor, in the Fires of Mount Doom, the Dark Lord Sauron forged a master ring, and into this ring he poured his cruelty, his malice and his will to dominate all life.\n\nOne ring to rule them all.\n\nOne by one, the free lands of Middle-Earth fell to the power of the Ring, but there were some who resisted. A last all

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =100, chunk_overlap = 20)
doc_txt = text_splitter.split_documents(text_documents)

In [39]:
doc_txt

[Document(metadata={'source': 'speech.txt'}, page_content='The world is changed.\n\nI feel it in the water.\n\n\nI feel it in the earth.\n\n\nI smell it in the air.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Much that once was is lost, for none now live who remember it.'),
 Document(metadata={'source': 'speech.txt'}, page_content='It began with the forging of the Great Rings. Three were given to the Elves, immortal, wisest and'),
 Document(metadata={'source': 'speech.txt'}, page_content='wisest and fairest of all beings. Seven to the Dwarf-Lords, great miners and craftsmen of the'),
 Document(metadata={'source': 'speech.txt'}, page_content='craftsmen of the mountain halls. And nine, nine rings were gifted to the race of Men, who above all'),
 Document(metadata={'source': 'speech.txt'}, page_content='Men, who above all else desire power. For within these rings was bound the strength and the will to'),
 Document(metadata={'source': 'speech.txt'}, page_content='and the 

In [40]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [41]:
embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [42]:
db = Chroma.from_documents(doc_txt, embeddings)

In [43]:
query = "Who are great miners?"
result = db.similarity_search(query)
result[0].page_content

'malice and his will to dominate all life.'

In [44]:
from langchain_ollama.llms import OllamaLLM

In [45]:
llm = OllamaLLM(model = "llama3.2")
llm

OllamaLLM(model='llama3.2')

In [46]:
from langchain_core.prompts import ChatPromptTemplate

In [47]:
prompt = ChatPromptTemplate.from_template("""
Answer the following questions based only on the provides context. 
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
           {context}                               
</context>
Question: {input}
""")

In [48]:
#Chain instruction

from langchain.chains.combine_documents import create_stuff_documents_chain

In [49]:
document_chain = create_stuff_documents_chain(llm,prompt)

In [50]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x11a8703e0>, search_kwargs={})

In [51]:
## Retrieval chain

from langchain.chains import create_retrieval_chain

In [52]:
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [53]:
response = retrieval_chain.invoke({"input": "Who are great miners?"})

In [54]:
response['answer']

"I don't see any context about great miners in the provided text. The text appears to be a descriptive passage about a story or legend involving a person named Malice and a king. There is no mention of mining.\n\nIf you would like, I can try to answer another question based on the provided context. Alternatively, if you provide more information about the context for this specific question, I'll do my best to help."